<center><h1> COVID-19 social distancing simulation </h1></center>

<h1><center>When can we resume our normal lives (Germany scenario)?</h1></center>

## Acknowledgements

Insired from **Element AI** team's *corona-calculator* (https://corona-calculator.herokuapp.com/) and **Christian Hubbs**'s dynamic diffusion models for epidemiology (https://www.datahubbs.com/)

## Introduction

The purpose of this analysis is to make best "guess" on when is the time for us to get back to our normal lives in **Germany**.

Robert Koch Institute (RKI)'s estimation of "no social-distancing scenario" for **Germany** was used here as a benchmark for social distancing simulation. The report stated that the number of infected cases will rise to **~10 millions** in 3 months if no intervention is implemented (as of 19 Mar 2020). (https://www.iamexpat.de/expat-info/german-expat-news/rki-coronavirus-could-infect-10-million-people-germany)

Given the fact that society is now in a "lock-downed", I would like to create 5 different scenarios of social distancing using social demographics & disease data with simple epidemiological models in this first MVP.

The 5 scenarios are as follow:
 1. No social distancing at all
 2. 25% social distancing
 3. 50% social distancing
 4. 75% social distancing (which is the closest to reality)
 5. 100% social distancing (mimimum contact to any other human)
   


**Initialization**

In [1]:
from __future__ import print_function
%precision %.2f
%matplotlib inline
# Interactive widget for plot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import itertools

import graphic
import func

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from enum import Enum

# Plotly
import plotly.express as px
import plotly.graph_objects as go
plt.rcParams["figure.figsize"] = (18, 10)  # plot size

import datetime
from datetime import datetime, timedelta

# # interactive dataframe
import itables.interactive
from itables import show

# dplyr-style for python
from dppd import dppd
dp, X = dppd()

# display all results from cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

<IPython.core.display.Javascript object>

## Overview

The whole project can be quickly summarized in this chart:
![COVID19_DE_chart](https://github.com/o0oBluePhoenixo0o/COVID_19-Social-Dist-Simulation/blob/master/img/COVID%2019%20DE%20simulation.png?raw=true)



### Inputs

#### MIDAS research networks: (https://github.com/midas-network/COVID-19)
 - Fetch data directly from MIDAS repository to get latest estimated parameters from all related researches on COVID-19.
     - **Incubation period** - time elapsed between exposure and when symptoms and signs are first apparent
     - **Recovery rate** - "time from symptom onset to recovery", obtained from Singapore and China research journals (using lognormal parametric survival methods & ratio of cumulative number of recovered/deaths and that of infected at time *t*)
     - **Transmission rate** & **Basic reproduction rate (R0)** - the average number of people who will catch a disease from one contagious person. It specifically applies to a population of people who were previously free of infection and haven’t been vaccinated. If a disease has an R0 of 18, a person who has the disease will transmit it to an average of 18 other people, as long as no one has been vaccinated against it or is already immune to it in their community.
     One way to calculate R0 is:

       **R0 = Probability of transmission x Number of Contacts per day x Number of infectious days**


![R0 example](https://miro.medium.com/max/648/1*kc4-Bv2nzIvb9xG6ELHuzA.png)
<i><center>Increasing R0 values indicate more infectious diseases (Source: [Healthline](https://www.healthline.com/health/r-nought-reproduction-number))</center></i>

In [28]:
# estimated parameters 
est_para = pd.read_csv('https://raw.githubusercontent.com/midas-network/COVID-19/master/parameter_estimates/2019_novel_coronavirus/estimates.csv')

# Incubation period
IB = (dp(est_para)
         .query('name == "incubation period"')
         .select(['peer_review','id','value_type','data_description','end_date','upper_bound','lower_bound','value'])
         .mutate(value = pd.to_numeric(X['value'], errors='coerce'))
        .pd)
IB = round(IB.value.mean(),2)

# Basic reproduction value
R0 = (dp(est_para)
         .query('abbreviation == "R0"')
         .select(['peer_review','id','abbreviation','data_description','end_date','upper_bound','lower_bound','value'])
         .mutate(value = pd.to_numeric(X['value'], errors='coerce'))
        .pd)
R0 = round(R0.value.mean(),2)

# Transmission rate
TR = (dp(est_para)
     .query("name == 'transmission rate'")
     .select(['id','location_type','abbreviation','name','end_date','upper_bound','lower_bound','value'])
      .mutate(value = pd.to_numeric(X['value'], errors='coerce'))
    .pd)
TR = round(TR.value.max()/100,2)

# Recovery time
RT = (dp(est_para)
         .query('name == "time from symptom onset to recovery"') # days
         .select(['peer_review','id','value_type','data_description','end_date','upper_bound','lower_bound','value'])
         .mutate(value = pd.to_numeric(X['value'], errors='coerce'))
        .pd)
RT = round(RT.value.mean(),2)

# Death rate of critically ill patients who don't have access to a hospital bed.
# This is the max reported from Wuhan:
# https://wwwnc.cdc.gov/eid/article/26/6/20-0233_article
CDR = 0.122

# Display result
print('Incubation period: ' + str(IB))
print('R0 - basic reproduction rate: ' + str(R0))
print('Transmission rate: ' + str(TR))
print('Recovery time: '+ str(RT))
print('Critical death rate: ' + str(CDR))

Incubation period: 6.55
R0 - basic reproduction rate: 3.26
Transmission rate: 0.01
Recovery time: 21.02
Critical death rate: 0.122


#### JHU (Johns Hopkins University) and RKI (Robert Koch Institute) repositories
 - Number of *Confirmed / Recovered / Death* world data is obtained from Johns Hopkins University's repository (https://github.com/CSSEGISandData/COVID-19/)- this data is only contained the numbers on country-level.
  
 - State-level numbers from RKI (https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0)

Getting data from source (2) JHU + RKI

In [3]:
# Get total timeseries from JHU
global_confirmed = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
global_death = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [4]:
# Rename columns
global_confirmed = global_confirmed.rename(columns={'Country/Region':'Country'})
global_recovered = global_recovered.rename(columns={'Country/Region':'Country'})
global_death = global_death.rename(columns={'Country/Region':'Country'})

target_country = 'Country == "Germany"'
# Get the latest data
target_date = func.get_latest_date(global_confirmed,
                                   global_recovered,
                                   global_death)
# prepare historical dataset 
historical_df = func.prepare_historical_df(target_country,
                                           target_date,
                                           global_confirmed,
                                           global_recovered,
                                           global_death)

no_d = func.get_cases_number(target_date,target_country,
                             global_confirmed,global_recovered,global_death)[0]
no_c = func.get_cases_number(target_date,target_country,
                             global_confirmed,global_recovered,global_death)[1]
no_r = func.get_cases_number(target_date,target_country,
                             global_confirmed,global_recovered,global_death)[2]

Latest cases data is captured on 3/28/20


#### Social demographics data
 - Age distribution for **Germany** - pyramid age (https://www.populationpyramid.net/germany/2019/). The other countries data can also be collected at this page (up to 2018 for some countries)
 - Number of available hospital beds and health care staffs from OECD databank (https://data.oecd.org/healtheqt/hospital-beds.htm)
 - For mortality rates per age cohort, I use data from ["Imperial College COVID-19 Response Team"](https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-NPI-modelling-16-03-2020.pdf) report on 16 Mar 2020. The table below shows the mortality rates for each age cohorts:
 ![Mortality Rate](https://github.com/o0oBluePhoenixo0o/COVID_19-Social-Dist-Simulation/blob/master/img/mortality_rate_1603.PNG?raw=true)
 <center><i>Source: Imperial College COVID-19 Response Team</i></center>

In [5]:
# Proportion of DE is taken from the latest age pyramid: https://www.populationpyramid.net/germany/2019/
AGE_DATA = pd.read_csv("./Data/age_data.csv", index_col="Age Group")
BED_DATA = func.preprocess_bed_data("./Data/OECD hospital beds.csv")
country_data = pd.read_csv("./Data/OECD demographics.csv")

# Only filter out total population data
country_data = (dp(country_data)
               .query("VAR == 'DEMODOMP' & UNIT == 'EFFPEREF' & Year == 2018")
               .mutate(Value = X.Value * 1000)
               .select(["Country","Value"])
               .pd)

population = (dp(country_data)
             .query(target_country)
             .select('Value')
             .pd).iloc[0]['Value']

num_hospital_beds = (dp(BED_DATA)
                     .query(target_country)
                     .select('Latest Bed Estimate')
                     .pd).iloc[0]['Latest Bed Estimate']

### Models


#### SIR

The model asummes:
   - The population size is fixed (i.e., no births, deaths due to disease, or deaths by natural causes)
   - Incubation period of the infectious agent is instantaneous
   - Duration of infectivity is same as length of the disease
   - Completely homogeneous population with no age, spatial, or social structure
    
![SIR Model](https://upload.wikimedia.org/wikipedia/commons/8/8a/SIR.PNG)
![SIR func](http://idmod.org/docs/general/_images/math/7edd99664ee58dde174cfe47bf51ade942786541.png)

   **Where N (population) = S(Susceptible) + I (Infected)+ R (Recovered)*   
   
   The crucial factor governing disease spread is R0 (the basic reproduction rate), which is the **average number of people somebody with the disease infects.**
   
   The parameters , **β** (beta) and **γ** (gamma) are defined as follow:

   1. **β** = average contact rate in the population.
   
   Another way to defined **β** is:
    - **β** = Probability of transmission x Number of contacts
   
   2. **γ** = inverse of the mean infectious period (1/t_infectious). Or usually known as "recovery rate"
   
And R0 can be calculated to get those parameters:

<b><center>R0 = β/γ</center></b>


#### SEIR + vital
![SEIR Model](https://upload.wikimedia.org/wikipedia/commons/3/3d/SEIR.PNG)
![SEIR func](http://idmod.org/docs/general/_images/math/5c34ba7654b6b1031ac83c60ea98007456d22ee3.png)

**Where N (population) = S(Susceptible) + E (Exposed) + I (Infected)+ R (Recovered)** 
   
   The 3rd parameter **δ** (delta) is calculated as follow:
   
   3. **δ** = inverse of the incubation period (1/t_incubation)
      
   Here we have another parameter to show those who are exposed (asymptomatic)
With vital dynamics (birth + death rate **μ** and **ν**)

![SEIR func vital](http://idmod.org/docs/general/_images/math/7a0619d75a08582ad67f21d3a0ffb938b8576920.png)
   
   - **μ** and **ν** represent the birth and death rates, respectively, and are assumed to be equal to maintain a constant population
   
#### SEIR + DH + vital
   - Here I include another 2 factors D & H which account for number of death cases and number of hospitalized cases. The 2 numbers are calculated as follow:
   

## Simulation

### Forecasting

In [23]:
# Define parameters
t_max = 100
# dt = .1
t = np.linspace(0, t_max, int(t_max/dt) + 1)
N = population

init_vals = (N - (no_c + no_r + no_d),  # S
             (no_c + no_r + no_d), # E
             no_c,  # I
             no_r + no_d) #R

# Parameters
# inverse of incubation period
delta = 1/IB
# Beta = gamma - (recovery rate =  1/infectious_day) * R0 
beta = 1/RT * R0 
# Recovery rate
gamma = 1/RT
social_dist = 1 # no social dist

params = delta, beta, gamma, social_dist

# Run simulation
results = seir_model_with_soc_dist(init_vals, params, t)
results = pd.DataFrame(results,columns = ['Susceptible','Exposed','Infected','Recovered'])

In [24]:
1 - (no_c + no_r + no_d)/N*100
(no_c + no_r + no_d)/N*100
no_c/N*100
(no_r + no_d)/N*100

results = (dp(results)
 .mutate(Susceptible = X.Susceptible)
 .mutate(Exposed = X.Exposed)
 .mutate(Infected = X.Infected)
 .mutate(Recovered = X.Recovered)
.pd)
results['id'] = results.index
results= pd.melt(results,
                 id_vars=['id'], 
                 value_vars=['Susceptible','Exposed','Infected','Recovered'])
results.head(100)

0.9195469632478344

0.08045303675216565

0.06968634802228224

0.01076668872988342

id,variable,value


In [25]:
fig = px.line(results, x="id", y="value", color="variable", template='plotly_white')
    
fig.layout.update(
    xaxis_title="Date",
    font=dict(family="Arial", size=12))
    
fig

Layout({
    'font': {'family': 'Arial', 'size': 12},
    'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
    'margin': {'t': 60},
    'template': '...',
    'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
    'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}
})

In [128]:
def f(contact_rate):
    return(contact_rate)

w = interactive(f, contact_rate= widgets.IntSlider(min=0, max=100, step=1, value=12))
print('How many people does an infected individual meet daily?')
display(w)

How many people does an infected individual meet daily?


interactive(children=(IntSlider(value=12, description='contact_rate'), Output()), _dom_classes=('widget-intera…

In [ ]:
    
true_cases_estimator = TrueInfectedCasesModel(ReportingRate.default)
estimated_true_cases = true_cases_estimator.predict(number_cases_confirmed)
asymptomatic_cases_estimator = AsymptomaticCasesModel(AsymptomaticRate.default)

sir_model = SIRModel(
        transmission_rate_per_contact=TransmissionRatePerContact.default,
        contact_rate = w.result,
        recovery_rate = RecoveryRate.default,
        normal_death_rate = MortalityRate.default,
        critical_death_rate = CriticalDeathRate.default,
        hospitalization_rate = HospitalizationRate.default,
        hospital_capacity = num_hospital_beds)

df = get_predictions(
    cases_estimator=true_cases_estimator,
    sir_model=sir_model,
    num_diagnosed=number_cases_confirmed,
    num_recovered=number_cases_recovered,
    num_deaths= number_cases_deaths,
    area_population=population)

### Plotting

In [ ]:
# Historical data
graphic.plot_historical_data(historical_df)


In [ ]:

# Forecast results
df_base = df[~df.Status.isin(["Need Hospitalization"])]
df_base['Date'] = None
for idx,row in df_base.iterrows():
    df_base['Date'][idx] = datetime.strftime(datetime.now() + timedelta(int(df_base['Days'][idx])),"%d/%m/%y")
df_base['Date'] = pd.to_datetime(df_base['Date'],format="%d/%m/%y")
graphic.infection_graph(df_base, 50000000)
# graphic.infection_graph(df_base, df_base.Forecast.max())

# Combination of historical + forecast results
df_full = (dp(df_base)
           .select("-Days")
           .rename(columns = {'Forecast':'Number'})
           .append(historical_df)
           .pd)
print('Combination historical and forecast COVID-19 cases')
graphic.plot_historical_data(df_full)

# Forecast deathrates per age groups
num_dead = df[df.Status == "Dead"].Forecast.iloc[-1]
num_recovered = df[df.Status == "Recovered"].Forecast.iloc[-1]
outcomes_by_age_group = get_status_by_age_group(AGE_DATA, MortalityRate.default,
                                                num_dead, num_recovered)
fig = graphic.age_segregated_mortality(
    outcomes_by_age_group.loc[:, ["Dead", "Need Hospitalization"]])

fig


# Hospital occupancy rate
peak_occupancy = df.loc[df.Status == "Need Hospitalization"]["Forecast"].max()

num_beds_comparison_chart = graphic.num_beds_occupancy_comparison_chart(
    num_beds_available=num_hospital_beds, 
    max_num_beds_needed=peak_occupancy)
num_beds_comparison_chart

## Extra plots

In [ ]:
# Read DE data from RKI's repository
DE_data = pd.read_csv('https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv')
test = (dp(DE_data)
 .select(['Meldedatum','Bundesland','Landkreis',
         'AnzahlFall','AnzahlTodesfall'])
 .groupby(['Meldedatum','Bundesland'])
 .sum()
 .reset_index()
 .pd)
test['AnzahlFall_cumulative'] = test.groupby(['Bundesland'])['AnzahlFall'].apply(lambda x: x.cumsum())
test['AnzahlTodesfall_cumulative'] = test.groupby(['Bundesland'])['AnzahlTodesfall'].apply(lambda x: x.cumsum())

fig = px.line(test, x="Meldedatum", y = "AnzahlFall_cumulative", color="Bundesland", template='plotly_white')
fig.layout.update(
        title = "Total infected cases per state in Germany",
        xaxis_title="Date",
        yaxis_title="Total infected case",
        font=dict(family="Arial", size=12))
fig

In [ ]:
# Set constant values

# """
# SEIR model constants
# """
class RecoveryRate:
    default = 1 / 10  # Update with MIDAS network everytime running the code

class MortalityRate:
    # Take weighted average of death rate across age groups. This assumes each age group is equally likely to
    # get infected, which may not be exact, but is an assumption we need to make for further analysis,
    # notably segmenting deaths by age group.
    default = (AGE_DATA.Proportion_DE_2020 * AGE_DATA.Mortality).sum()

class CriticalDeathRate:
    # Death rate of critically ill patients who don't have access to a hospital bed.
    # This is the max reported from Wuhan:
    # https://wwwnc.cdc.gov/eid/article/26/6/20-0233_article
    default = 0.122

class SymptomState(Enum):
    ASYMPTOMATIC = "asymptomatic"
    SYMPTOMATIC = "symptomatic"
    
class TransmissionRatePerContact:
    # Probability of a contact between carrier and susceptible leading to infection.
    # Using the mean value of all research reports
    default = round(TR,2)
#     default = 0.018
    
    # The transmission rate of a asymptomatic infected individual is lower by a certain ratio
    # The ratio is reported to be 55%
    # source: https://science.sciencemag.org/content/early/2020/03/13/science.abb3221
    default_per_symptom_state = {
        SymptomState.ASYMPTOMATIC : 0.55 * default,
        SymptomState.SYMPTOMATIC : default,
    }

class AverageDailyContacts:
    min = 0
    max = 50
    default = 15
    
class AsymptomaticRate:
    # Proportion of true cases showing no symptoms
    # The number comes from a study led on passengers of the Diamond Princess Cruise, in Japan
    # https://www.eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.10.2000180
    default = 0.179
    
# """
# Health care constants
# """

class ReportingRate:
    # Proportion of true cases diagnosed
    # default = 0.14
    default = 1 # assuming all reported cases are true case

class HospitalizationRate:
    # Cases requiring hospitalization. We multiply by the ascertainment rate because our source got their estimate
    # from the reported cases, whereas we will be using it with total cases.
    default = 0.19 * ReportingRate.default